In [1]:
## Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## Set path
import os
os.chdir('/content/drive/MyDrive/ADM_HW5')

In [12]:
## Import Utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import math
from tqdm import tqdm
from datetime import datetime, date
from random import sample

In [4]:
## Load Data
with open('sx-stackoverflow-a2q.txt') as f:
    a2q = f.readlines()
    f.close()

In [5]:
## Load Data
with open('sx-stackoverflow-c2q.txt') as f:
    c2q = f.readlines()
    f.close()

In [6]:
## Load Data
with open('sx-stackoverflow-c2a.txt') as f:
    c2a = f.readlines()
    f.close()

In [7]:
## Init Graph
G = nx.DiGraph()

In [8]:
## Merge Graph
t_start = datetime(2009, 10, 1)
t_end = datetime(2010, 10, 1)

for line in a2q:
    line = line.replace('\n', '')
    line = line.split(' ') 
    t = datetime.fromtimestamp(int(line[2]))
    t_truncated = date(t.year,t.month, t.day)
    if t_start <= t < t_end :
        if G.has_edge(line[0], line[1]) :
            G[line[0]][line[1]]['weight'] += 0.6
            G[line[0]][line[1]]['timestamp'].append(t_truncated)
        else :
            G.add_edge(line[0], line[1], weight=0.6)
            G[line[0]][line[1]]['timestamp']=[t_truncated]

for line in c2q:
    line = line.replace('\n', '')
    line = line.split(' ') 
    t = datetime.fromtimestamp(int(line[2]))
    t_truncated = date(t.year,t.month, t.day)
    if t_start <= t < t_end :
        if G.has_edge(line[0], line[1]) :
            G[line[0]][line[1]]['weight'] += 0.3
            G[line[0]][line[1]]['timestamp'].append(t_truncated)
        else :
            G.add_edge(line[0], line[1], weight=0.3)
            G[line[0]][line[1]]['timestamp']=[t_truncated]

for line in c2a:
    line = line.replace('\n', '')
    line = line.split(' ') 
    t = datetime.fromtimestamp(int(line[2]))
    t_truncated = date(t.year,t.month, t.day)
    if t_start <= t < t_end :
        if G.has_edge(line[0], line[1]) :
            G[line[0]][line[1]]['weight'] += 0.1
            G[line[0]][line[1]]['timestamp'].append(t_truncated)
        else :
            G.add_edge(line[0], line[1], weight=0.1)
            G[line[0]][line[1]]['timestamp']=[t_truncated]

### **Functionality 2**

In [9]:
def shortest_path(G, source, target):
    unvisited = list(nx.nodes(G)) ## all nodes
    dist = {node : float('infinity') for node in list(nx.nodes(G))} ## distances from each node to source
    previous = {node : None for node in list(nx.nodes(G))} ## previous node  
    dist[source] = 0 ## The distance of the source node is 0
    
    while unvisited :  
        dist_min = math.inf
        for node in unvisited:
            if dist[node] <= dist_min :
                dist_min = dist[node]
                u = node
        unvisited.remove(u) ## removing the node from the list of the unvisited nodes
        
        ## if the visited node is the target stop the visits
        if u == target:
            break
            
        ## Examine its neighbours
        for neighbor in G.neighbors(u):
            new_dist = dist[u] + G[u][neighbor]['weight']
            if dist[neighbor] > new_dist:
                ## update the shortest distance
                dist[neighbor] = new_dist
                ## update the previous node
                previous[neighbor] = u
                
    if previous[target] == None:
        return ('Not Connected',[])
    
    path = [target]
    while source not in path:
        path.append(previous[path[-1]])
    path.reverse()

    return dist[target], path # return the distance

In [ ]:
def PageRank(G, alpha, node, weight = 'weight'):
    # Create a copy of the graph where the sum of the weights of the edges
    # of each node is equal to one
    W = nx.stochastic_graph(G, weight = weight)
    N = len(G) # Number of nodes

    # Initialize a dictionary of ones / N
    q = dict.fromkeys(W, 1.0/N)

    # Store the starting dictionary
    start_q = q

    # List of nodes with zero edges that point out to them
    nodes = set()
    aa = set()
    for n in W:
        nodes.add(n)
        for _, ign in W.edges(n):
            aa.add(ign)
    ignored_nodes = nodes.difference(aa)

    # Power Iteration: make up to 100 iterations
    for _ in range(100):
        qlast = q # Probability to be at a certain state after t-1 steps
        qt = dict.fromkeys(qlast.keys(), 0) 
        # Score for the ignored nodes
        # Only the damping parameter is considered
        ignored_sum = alpha * sum(qlast[n] for n in ignored_nodes)
        
        for n in qt: # For each node
            # Extract the destination node of the edges and the weight of the edge
            for _, dest_node, wt in W.edges(n, data = weight):
                # Score of the destination node = alpha*q(t-1)
                qt[dest_node] += alpha * qlast[n] * wt 
            # Score of each node = q[0]*alpha + (1 - alpha)*q[0]
            # Total probability to get at the nodes with damping or from another node
            qt[n] += ignored_sum * start_q.get(n, 0) + (1.0 - alpha) * start_q.get(n, 0)
        
        # Check convergence using l1 norm
        err = sum([abs(qt[n] - qlast[n]) for n in qt])
        # Fix tolerance value to check convergence at 0.0001
        if err < N * 0.0001: 
            return qt[node] # Return the PageRank score for the requested node

In [ ]:
def Closeness(G, u):
    '''
    Input:
        - G: a graph
        - u: a node of the graph
    '''
    # Initialize the sum for the shortest path distances
    s = 0

    for node in tqdm(G.nodes):
        # Compute the shortest path for each node
        dist = shortest_path(G, u, node)
        # print(dist)
        # If a path exists add its distance to the sum
        if (dist[0] != 'Not Connected'):
            s += int(dist[0])

    # Compute the closeness centrality score
    return ((len(G) - 1)/ s)
    

In [ ]:
def funct_2(node, time_start, time_end, metric):
    '''
    Input:
        node <- a user/node
        time_start <- start of the interval of time
        time_end <- end of the interval of time
        metric <- one of the following metrics:
                    1. Betweenness
                    2. PageRank
                    3. Closeness Centrality
                    4. Degree Centrality
    '''
    subG = nx.DiGraph()
    edges = list(G.edges)
    for edge in edges:
        if time_start <= G.edges[edge[0], edge[1]]['timestamp'][0] < time_end:
            subG.add_edge(edge[0], edge[1])
            subG[edge[0]][edge[1]]['weight'] = G.edges[edge[0], edge[1]]['weight']
            subG[edge[0]][edge[1]]['timestamp'] = G.edges[edge[0], edge[1]]['timestamp']

    sub_edges = list(subG.edges)

    if metric == 1: # Betweenness centrality
        print('') 
    
    elif metric == 2: # PageRank
        pr = PageRank(subG, 0.25, node)
        print(f'PageRank score of {node}:', pr)

    elif metric == 3: # Closeness centrality
        cc = Closeness(subG, node)
        print(f'Closeneess centrality of {node}', cc) 

    elif metric == 4: # Degree centrality
        count = 0
        for edge in sub_edges:
            if edge[0] == str(node) or edge[1] == str(node):
                count += 1
            if edge[0] == edge[1] == str(node):
                count -= 1

        degree_centrality = count / (len(subG) - 1)
        print(f'Degree centrality of {node}:', degree_centrality)
    
    else:
        print('Insert a valid metric!')

In [10]:
time_start = date(2010, 1, 1)
time_end = date(2010, 1, 3)
subG = nx.DiGraph()
for edge in list(G.edges):
    if time_start <= G.edges[edge[0], edge[1]]['timestamp'][0] < time_end:
        subG.add_edge(edge[0], edge[1])
        subG[edge[0]][edge[1]]['weight'] = G.edges[edge[0], edge[1]]['weight']
        subG[edge[0]][edge[1]]['timestamp'] = G.edges[edge[0], edge[1]]['timestamp']

In [ ]:
nx.pagerank(subG, 0.25)['140740']

0.0002393118371791511

In [ ]:
PageRank(subG, 0.25, '140740')

0.00024123412649959677

In [ ]:
nx.closeness_centrality(subG, '140740')

0.05490359465936323

In [ ]:
Closeness(subG, '140740')

  0%|          | 0/3451 [00:00<?, ?it/s]

('Not Connected', [])
(0.6, ['140740', '211662'])
(0.6, ['140740', '226834'])
(1.2, ['140740', '138604'])
(0.6, ['140740', '223541'])


  0%|          | 6/3451 [00:00<02:36, 21.99it/s]

(4.3999999999999995, ['140740', '138604', '192525', '19068', '238623', '187606', '55589', '21240', '209605', '179235', '22656'])
(5.3, ['140740', '138604', '192525', '19068', '238623', '187606', '229824', '241467', '61974', '241671', '23897', '89521'])
(3.6, ['140740', '138604', '192525', '19068', '238623', '187606', '236370', '240698'])


  0%|          | 9/3451 [00:00<06:58,  8.23it/s]

...

100%|█████████▉| 3450/3451 [24:16<00:00,  2.51it/s]

('Not Connected', [])


100%|██████████| 3451/3451 [24:16<00:00,  2.37it/s]

('Not Connected', [])


0.6134423897581792

In [11]:
nx.betweenness_centrality(subG)['140740']

1.1188490174627073e-05

In [18]:
def Beetweenness(G, node):
    '''
    Input:
        - G: a graph
        - node: a node of the graph G
    '''
    s = 0      # Initialize the sum for all the shortest paths found
    s_node = 0 # Initialize the sum for all the shortest paths that contains node

    sample_nodes = sample(list(G.nodes), 100)

    if node not in sample_nodes:
        sample_nodes.append(node)

    for n1 in tqdm(sample_nodes):
        print(n1)
        for n2 in sample_nodes:
            dist = shortest_path(G, n1, n2)
            if(len(dist[1]) != float('inf') or len(dist[1]) != 0):
                s += 1
                if node in dist[1]:
                    s_node += 1
            else:
                print("There aren't shorthest paths here!")

    return s_node/s

In [19]:
Beetweenness(subG, '140740')

  0%|          | 0/101 [00:00<?, ?it/s]

241616


  1%|          | 1/101 [00:42<1:10:42, 42.42s/it]

1942


  2%|▏         | 2/101 [01:24<1:09:55, 42.38s/it]

...

242166


 99%|█████████▉| 100/101 [1:28:50<00:52, 52.29s/it]

140740


100%|██████████| 101/101 [1:29:33<00:00, 53.21s/it]


0.008528575629840211